In [2]:
import e3nn.o3 as o3
import torch
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.core.structure import Structure

from mp_api.client import MPRester
import numpy as np 
import pandas as pd
import spglib

# Initialize MPRester
with MPRester("zckQao2291DWQRcIyq96cAwvZH9DAdTy") as mpr:
    docs = mpr.materials.summary.search(material_ids=["mp-1434"])  # MoS2 Triagonal Crystal System
    primitive_pymatgen = docs[0].structure  # Primitive cell from Materials Project    
    sga = SpacegroupAnalyzer(primitive_pymatgen, symprec=1e-4)    
    primitive_sga = sga.get_refined_structure()
    refined_symmetry = SpacegroupAnalyzer(primitive_sga, symprec=1e-4)

# Define Cartesian basis for a rank-3 tensor (shift current tensor)
irreps = o3.Irreps("3x1o + 3x1o + 3x1o")  # Three Cartesian vector components (x, y, z)

# Create a tensor product for a rank-3 tensor
tensor_product = o3.FullTensorProduct(irreps, irreps)
print("tp \n", tensor_product)


# Generate a random 3x3x3 tensor
sigma_tensor = torch.randn(3, 3, 3)
# Symmetrize the tensor: Ensure sigma_{αβγ} = sigma_{αγβ}
sigma_tensor = 0.5 * (sigma_tensor + sigma_tensor.permute(0, 2, 1))
# Print the symmetrized tensor
print("Symmetrized Shift Current Tensor (σ_{αβγ} = σ_{αγβ}):")
print(sigma_tensor)

aux = torch.zeros([3,3,3])
for op in refined_symmetry.get_symmetry_operations(cartesian= True):
    R = op.rotation_matrix
    R = torch.tensor(R).to(torch.float64)
    # print(R,sigma_tensor)
    transformed_tensor = torch.einsum('ai,bj,ck,ijk->abc', R, R, R, sigma_tensor.to((torch.float64)))
    # print(transformed_tensor)
    aux += transformed_tensor
print(aux/len(refined_symmetry.get_symmetry_operations(cartesian= True)))


ValueError: unconverted data remains: .post